### Задача:

 по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята. 

### План решения:

1. Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

In [140]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [141]:
df = pd.read_csv('grant_data_imb.csv')
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
Grant.Status,0,0,0,0,0,1,0,0,0,0
Sponsor.Code,97A,36D,317A,62B,1A,4D,2B,2B,4D,60D
Grant.Category.Code,30B,10A,30D,10B,10A,10A,10A,10A,10A,30D
Contract.Value.Band...see.note.A,A,G,NaN,B,NaN,F,NaN,D,NaN,NaN
RFCD.Code.1,321024.0,300201.0,321013.0,321103.0,270603.0,321015.0,240301.0,260109.0,320702.0,NaN
RFCD.Percentage.1,50.0,100.0,100.0,30.0,60.0,80.0,90.0,90.0,100.0,NaN
RFCD.Code.2,321013.0,0.0,0.0,321105.0,321205.0,329903.0,240204.0,260301.0,0.0,NaN
RFCD.Percentage.2,30.0,0.0,0.0,40.0,30.0,10.0,10.0,10.0,0.0,NaN
RFCD.Code.3,291502.0,0.0,0.0,321204.0,320603.0,320503.0,0.0,0.0,0.0,NaN
RFCD.Percentage.3,20.0,0.0,0.0,30.0,10.0,10.0,0.0,0.0,0.0,NaN


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

2. Подготовьте данные к обучению моделей:

a. Отделите целевую переменную Grant.Status;

In [143]:
features = df.drop(['Grant.Status', 'Person.ID.1'], axis=1)
target = df['Grant.Status']

b. Заполните пропуски

i. в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

ii. в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

In [144]:
features.describe().T

,count,mean,std,min,25%,50%,75%,max
RFCD.Code.1,3853.0,314904.682845,47163.318702,210000.0,280401.0,320801.0,321202.0,999999.0
RFCD.Percentage.1,3853.0,74.696860,26.875419,5.0,50.0,80.0,100.0,100.0
RFCD.Code.2,3853.0,161386.717104,161577.090361,0.0,0.0,240202.0,320702.0,440207.0
RFCD.Percentage.2,3853.0,17.642616,19.259007,0.0,0.0,10.0,30.0,90.0
RFCD.Code.3,3853.0,96437.197508,148599.260202,0.0,0.0,0.0,270208.0,440207.0
RFCD.Percentage.3,3853.0,7.089541,11.937533,0.0,0.0,0.0,15.0,70.0
RFCD.Code.4,3853.0,6835.177005,45889.060627,0.0,0.0,0.0,0.0,440105.0
RFCD.Percentage.4,3853.0,0.442512,3.228834,0.0,0.0,0.0,0.0,70.0
RFCD.Code.5,3853.0,1767.989878,23598.311088,0.0,0.0,0.0,0.0,419999.0
RFCD.Percentage.5,3853.0,0.128471,1.837888,0.0,0.0,0.0,0.0,40.0


In [145]:
for col in ['RFCD.Code.1', 'RFCD.Percentage.1', 'RFCD.Code.2', 'RFCD.Percentage.2', 'RFCD.Code.3',\
            'RFCD.Percentage.3', 'RFCD.Code.4', 'RFCD.Percentage.4', 'RFCD.Code.5', 'RFCD.Percentage.5',\
            'SEO.Code.1', 'SEO.Percentage.1', 'SEO.Code.2', 'SEO.Percentage.2', 'SEO.Code.3', 'SEO.Percentage.3',\
            'SEO.Code.4', 'SEO.Percentage.4', 'SEO.Code.5', 'SEO.Percentage.5', 'Dept.No..1',\
            'Faculty.No..1','Number.of.Successful.Grant.1', 'Number.of.Unsuccessful.Grant.1',\
            'A..1', 'A.1', 'B.1', 'C.1']:
    features.fillna({col: 0}, inplace=True)

In [146]:
features.fillna({'Year.of.Birth.1': df['Year.of.Birth.1'].mean().round(0)}, inplace=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            3856 non-null   object 
 1   Grant.Category.Code                     3856 non-null   object 
 2   Contract.Value.Band...see.note.A        1953 non-null   object 
 3   RFCD.Code.1                             4113 non-null   float64
 4   RFCD.Percentage.1                       4113 non-null   float64
 5   RFCD.Code.2                             4113 non-null   float64
 6   RFCD.Percentage.2                       4113 non-null   float64
 7   RFCD.Code.3                             4113 non-null   float64
 8   RFCD.Percentage.3                       4113 non-null   float64
 9   RFCD.Code.4                             4113 non-null   float64
 10  RFCD.Percentage.4                       4113 non-null   floa

In [147]:
features.describe(include="object").T

,count,unique,top,freq
Sponsor.Code,3856,226,4D,1006
Grant.Category.Code,3856,13,10A,2050
Contract.Value.Band...see.note.A,1953,16,A,961
Role.1,4067,7,CHIEF_INVESTIGATOR,3640
Country.of.Birth.1,3447,10,Australia,2530
Home.Language.1,394,2,English,316
With.PHD.1,2332,1,Yes,2332
No..of.Years.in.Uni.at.Time.of.Grant.1,3354,5,>=0 to 5,1337


In [148]:
features['With.PHD.1'].unique()

array(['Yes ', nan], dtype=object)

In [149]:
features['With.PHD.1'] = features['With.PHD.1'].apply(lambda x: 1 if x == 'Yes ' else 0)
features['With.PHD.1'].value_counts()

With.PHD.1
1    2332
0    1781
Name: count, dtype: int64

In [150]:
for col in ['Sponsor.Code', 'Grant.Category.Code', 'Contract.Value.Band...see.note.A',
                  'Role.1', 'Country.of.Birth.1']:
    features.fillna({col: 'Unknown'}, inplace=True)

In [151]:
features.groupby('Country.of.Birth.1')['Home.Language.1'].value_counts()

Country.of.Birth.1      Home.Language.1
Asia Pacific            Other               13
                        English             10
Australia               English            209
                        Other                3
Eastern Europe          Other                8
                        English              7
Great Britain           English             56
Middle East and Africa  Other                7
                        English              1
North America           English             13
                        Other                1
The Americas            English              9
                        Other                8
Unknown                 Other                3
                        English              1
Western Europe          Other               35
                        English             10
Name: count, dtype: int64

In [152]:
features[features['Home.Language.1'].isna()]['Country.of.Birth.1'].unique()

array(['Eastern Europe', 'Australia', 'Great Britain', 'North America',
       'Western Europe', 'Asia Pacific', 'Unknown', 'New Zealand',
       'Middle East and Africa', 'South Africa', 'The Americas'],
      dtype=object)

In [153]:
countries_dict = {
    'Australia':'English',
    'Great Britain':'English',
    'North America':'English',
    'New Zealand':'English',
    'Middle East and Africa':'English',
    'The Americas':'English',
    'Unknown':'Other',
    'South Africa':'Other',
    'Eastern Europe':'Other',
    'Western Europe':'Other',
    'Asia Pacific':'Other',
    'The Americas':'Other'
}

In [154]:
features['Home.Language.1'] = features.apply(lambda f: f['Home.Language.1'] if pd.notna(f['Home.Language.1'])\
                                 else countries_dict[f['Country.of.Birth.1']], axis=1)
features['Home.Language.1'].isna().sum()

np.int64(0)

In [155]:
features.groupby('No..of.Years.in.Uni.at.Time.of.Grant.1')['With.PHD.1'].value_counts()

No..of.Years.in.Uni.at.Time.of.Grant.1  With.PHD.1
>10 to 15                               1             359
                                        0              59
>5 to 10                                1             594
                                        0             145
>=0 to 5                                1             984
                                        0             353
Less than 0                             0             416
                                        1              80
more than 15                            1             282
                                        0              82
Name: count, dtype: int64

In [156]:
features[features['No..of.Years.in.Uni.at.Time.of.Grant.1'].isna()]['With.PHD.1'].value_counts()

With.PHD.1
0    726
1     33
Name: count, dtype: int64

In [157]:
features.fillna({'No..of.Years.in.Uni.at.Time.of.Grant.1':'Less than 0'}, inplace=True)
features['No..of.Years.in.Uni.at.Time.of.Grant.1'].isna().sum()

np.int64(0)

In [158]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4113 non-null   object 
 1   Grant.Category.Code                     4113 non-null   object 
 2   Contract.Value.Band...see.note.A        4113 non-null   object 
 3   RFCD.Code.1                             4113 non-null   float64
 4   RFCD.Percentage.1                       4113 non-null   float64
 5   RFCD.Code.2                             4113 non-null   float64
 6   RFCD.Percentage.2                       4113 non-null   float64
 7   RFCD.Code.3                             4113 non-null   float64
 8   RFCD.Percentage.3                       4113 non-null   float64
 9   RFCD.Code.4                             4113 non-null   float64
 10  RFCD.Percentage.4                       4113 non-null   floa

c. Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

In [159]:
categorical_columns = [ 
    'RFCD.Code.1', 
    'RFCD.Code.2', 
    'RFCD.Code.3', 
    'RFCD.Code.4', 
    'RFCD.Code.5', 
    'SEO.Code.1', 
    'SEO.Code.2', 
    'SEO.Code.3', 
    'SEO.Code.4', 
    'SEO.Code.5'
]

for col in categorical_columns:
    features[col] = features[col].astype(object)

In [160]:
features_ohe = pd.get_dummies(features, drop_first=True)
features_ohe.sample(7).T

,581,2212,3324,1710,3183,1092,3994
RFCD.Percentage.1,100.0,100.0,0.0,80.0,100.0,60.0,100.0
RFCD.Percentage.2,0.0,0.0,0.0,20.0,0.0,20.0,0.0
RFCD.Percentage.3,0.0,0.0,0.0,0.0,0.0,20.0,0.0
RFCD.Percentage.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RFCD.Percentage.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
Home.Language.1_Other,True,False,True,False,True,True,False
No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,False,True,False,False,False,False,False
No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,True,False,True,False,False,False,True
No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,False,False,False,False,False,False,False


In [161]:
for i in range(1,6):
    for col in features_ohe.columns:
        if col.startswith(f'RFCD.Code.{i}'):
            features_ohe[col] = features_ohe[col] * features_ohe[f'RFCD.Percentage.{i}']

        elif col.startswith('SEO.Code.{i}'):
            features_ohe[col] = features_ohe[col] * features_ohe[f'SEO.Percentage.{i}']

    features_ohe.drop(columns=[f'RFCD.Percentage.{i}'], inplace=True)
    features_ohe.drop(columns=[f'SEO.Percentage.{i}'], inplace=True)

features_ohe.head().T

,0,1,2,3,4
Year.of.Birth.1,1950.0,1955.0,1965.0,1960.0,1960.0
Dept.No..1,2563.0,1038.0,2763.0,2848.0,2678.0
Faculty.No..1,25.0,1.0,25.0,25.0,25.0
With.PHD.1,1,0,1,0,1
Number.of.Successful.Grant.1,2.0,0.0,4.0,1.0,5.0
...,...,...,...,...,...
Home.Language.1_Other,True,False,False,False,False
No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,False,False,True,False,True
No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,False,False,False,False,False
No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,False,True,False,True,False


d. Разделите данные на обучающую и тестовую части;

In [162]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_ohe, target, test_size=0.25, random_state=23
)

In [163]:
model = LogisticRegression(random_state=1, solver='liblinear', max_iter=1000)
model.fit(features_train, target_train)
predict_test = model.predict(features_valid)
print('Значение F1-метрики:', f1_score(predict_test, target_valid))

Значение F1-метрики: 0.458628841607565


e. Используйте масштабирование для получения признаков одинакового масштаба.

In [164]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_sc = scaler.transform(features_train)
features_valid_sc = scaler.transform(features_valid)

features_train_scaled = pd.DataFrame(features_train_sc, columns=features_train.columns)
features_valid_scaled = pd.DataFrame(features_valid_sc, columns=features_train.columns)

features_train_scaled.head().T

,0,1,2,3,4
Year.of.Birth.1,-1.925985,-0.835945,1.344135,-1.925985,-1.925985
Dept.No..1,0.372025,0.574505,0.391309,0.372025,0.338278
Faculty.No..1,0.310805,0.310805,0.310805,0.310805,0.310805
With.PHD.1,-1.114858,0.896975,0.896975,0.896975,0.896975
Number.of.Successful.Grant.1,0.021025,-0.661511,-0.661511,-0.661511,0.021025
...,...,...,...,...,...
Home.Language.1_Other,-0.593828,-0.593828,-0.593828,-0.593828,-0.593828
No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,-0.461238,-0.461238,-0.461238,-0.461238,-0.461238
No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,-0.694248,-0.694248,1.440408,-0.694248,-0.694248
No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,1.488573,-0.671784,-0.671784,-0.671784,-0.671784


In [165]:
model = LogisticRegression(random_state=1, solver='liblinear', max_iter=1000)
model.fit(features_train, target_train)
predict_test = model.predict(features_valid_scaled)
print('Значение F1-метрики:', f1_score(predict_test, target_valid))

Значение F1-метрики: 0.47619047619047616


Вывод: масштабирование немного улучшило значение метрики F1

3. Изучите распределение по целевой переменной, чтобы выяснить, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов;

In [166]:
display(target_train.value_counts())
display(target_valid.value_counts())

Grant.Status
0    2457
1     627
Name: count, dtype: int64

Grant.Status
0    802
1    227
Name: count, dtype: int64

Классы целевой переменной несбалансированы

In [167]:
def downsample(features, target, n_samples=None):
    # разделяем объекты различных классов
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # если не указано количество образцов, устанавливаем его равным количеству меньшего класса
    if n_samples is None:
        n_samples = min(len(features_zeros), len(features_ones))
    
    # случайным образом выбираем n_samples из более многочисленного класса
    features_zeros_downsampled = features_zeros.sample(n=n_samples, random_state=23)
    target_zeros_downsampled = target_zeros.sample(n=n_samples, random_state=23)
    
    # объединяем уменьшенные и неизмененные записи
    features_downsampled = pd.concat([features_ones, features_zeros_downsampled])
    target_downsampled = pd.concat([target_ones, target_zeros_downsampled])
    
    # перемешиваем объекты
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=23)
    
    return features_downsampled, target_downsampled

In [168]:
features_train_downsampled, target_train_downsampled = downsample(features_train_scaled, target_train.reset_index(drop=True))
features_val_downsampled, target_val_downsampled = downsample(features_valid_scaled, target_valid.reset_index(drop=True))

In [169]:
display(target_train_downsampled.value_counts())
display(target_val_downsampled.value_counts())

Grant.Status
1    627
0    627
Name: count, dtype: int64

Grant.Status
1    227
0    227
Name: count, dtype: int64

Классы целевой переменной сбалансированы

In [170]:
model = LogisticRegression(random_state=1, solver='liblinear', max_iter=1000)
model.fit(features_train_downsampled, target_train_downsampled)
predict_test = model.predict(features_val_downsampled)
print('Значение F1-метрики:', f1_score(predict_test, target_val_downsampled))

Значение F1-метрики: 0.6824034334763949


Вывод: устранение дисбаланса классов целевой переменной значительно улучшило значение F1-метрики 

4. Обучите модели и выберите лучшую:

a. Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;

In [171]:
model_cv = LogisticRegressionCV(solver='liblinear', random_state=12, class_weight='balanced', cv=5)
model_cv.fit(features_train_downsampled, target_train_downsampled)
roc_auc_score(target_val_downsampled, model_cv.predict_proba(features_val_downsampled)[:, 1])

np.float64(0.7405926759688719)

In [172]:
top_10_cv = pd.DataFrame(
    zip(list(features_ohe.columns), list(abs(model_cv.coef_[0]))), 
    columns=['feature', 'importance']
).sort_values(by=['importance'], ascending=False)
top_10_cv.head(10)

,feature,importance
264,Contract.Value.Band...see.note.A_Unknown,0.369906
5,Number.of.Unsuccessful.Grant.1,0.251543
235,Sponsor.Code_Unknown,0.180438
248,Grant.Category.Code_Unknown,0.180438
252,Contract.Value.Band...see.note.A_E,0.160683
251,Contract.Value.Band...see.note.A_D,0.142850
254,Contract.Value.Band...see.note.A_G,0.141499
4,Number.of.Successful.Grant.1,0.137233
253,Contract.Value.Band...see.note.A_F,0.116946
116,Sponsor.Code_24D,0.116288


b. Обучите модель случайного леса

i. Для подбора гиперпараметров и кросс-валидации используйте структуру GridSearchCV,

ii. Выберите наилучший вариант случайного леса и выведите его параметры,

iii. Оцените качество выбранной модели с помощью метрики rocauc,

iv. Выведите топ-10 признаков по важности. Используйте атрибут feature_importances_, чтобы узнать важность признаков в деревянных моделях.

In [173]:
# Обучение модели случайного леса с GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model_rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(features_train_downsampled, target_train_downsampled)

# Лучшие параметры
best_rf = grid_search.best_estimator_
print(f'Лучшие параметры случайного леса: {grid_search.best_params_}')

Лучшие параметры случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [174]:
roc_auc_score(target_val_downsampled, best_rf.predict_proba(features_val_downsampled)[:, 1], multi_class='ovr')

np.float64(0.905577441828873)

In [175]:
top_10_rf = pd.DataFrame(zip(list(features_ohe.columns), list(best_rf.feature_importances_)),
                                     columns=['feature', 'importance']
                                    ).sort_values(by=['importance'], ascending=False)
top_10_rf.head(10)

,feature,importance
264,Contract.Value.Band...see.note.A_Unknown,0.083632
5,Number.of.Unsuccessful.Grant.1,0.082402
4,Number.of.Successful.Grant.1,0.043971
1,Dept.No..1,0.027432
235,Sponsor.Code_Unknown,0.025846
7,A.1,0.024215
8,B.1,0.024080
6,A..1,0.023755
0,Year.of.Birth.1,0.023308
248,Grant.Category.Code_Unknown,0.022687


5. Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.

In [176]:
set(top_10_cv['feature'].head(10)) & set(top_10_rf['feature'].head(10))

{'Contract.Value.Band...see.note.A_Unknown',
 'Grant.Category.Code_Unknown',
 'Number.of.Successful.Grant.1',
 'Number.of.Unsuccessful.Grant.1',
 'Sponsor.Code_Unknown'}

### Вывод:

Согласно обеим моделям важны следующие признаки:
- 'Contract.Value.Band...see.note.A_Unknown',
- 'Grant.Category.Code_Unknown',
- 'Number.of.Successful.Grant.1',
- 'Number.of.Unsuccessful.Grant.1',
- 'Sponsor.Code_Unknown'

Сравнение величин ROC AUC показывает, что модель случайного леса (0.905577441828873) лучше подходит для данной задачи, чем логистическая регрессия (0.7405926759688719). Последняя имеет линейное предположение о зависимостях в данных, а случайный лес более гибок и может захватывать сложные взаимосвязи.